In [13]:
IMG_SIZE = 528
BATCH_SIZE = 16
EPOCHS = 50
VAL_SPLIT = 0.2

# Data Loading

In [11]:
import os

dataset_path = "../../data-collection/image-backend/saved_images"
hazard_path = os.path.join(dataset_path, "hazard/")
non_hazard_path = os.path.join(dataset_path, "non-hazard/")

In [ ]:
import tensorflow as tf

train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    labels="inferred",
    label_mode="int",
    image_size=(IMG_SIZE, IMG_SIZE),  # Resize to EfficientNetB6 input size
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_names=["hazard", "non-hazard"]  # Restrict to specific folders
)

ValueError: The `class_names` passed did not match the names of the subdirectories of the target directory. Expected: ['temp'], but received: ['Hazard', 'Non-hazard']

In [ ]:
from tensorflow.keras.applications.efficientnet import preprocess_input

train_ds = train_ds.map(lambda x, y: (preprocess_input(x), y))


val_ds = train_ds.take(int(VAL_SPLIT * len(train_ds)))
train_ds = train_ds.skip(int(VAL_SPLIT * len(train_ds)))

# Model Fine-tuning

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB6
import tensorflow as tf


def build_model():
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    model = EfficientNetB6(include_top=False, input_tensor=inputs, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(1, activation="sigmoid", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
    )
    return model

In [10]:
model = build_model()
model.summary()

165234480/165234480 [==============================] - 28s 0us/step


Model: "EfficientNet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 528, 528, 3)]        0         []                            
                                                                                                  
 rescaling_2 (Rescaling)     (None, 528, 528, 3)          0         ['input_2[0][0]']             
                                                                                                  
 normalization_1 (Normaliza  (None, 528, 528, 3)          7         ['rescaling_2[0][0]']         
 tion)                                                                                            
                                                                                                  
 rescaling_3 (Rescaling)     (None, 528, 528, 3)          0         ['normalization_1[0

In [ ]:
model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, verbose=2)

In [ ]:
model.save("../models/EfficientNET.keras")